In [1]:
import urllib2
import sqlite3
import numpy as np
import pandas as pd
from sys import stdout
from datetime import datetime

foursquaredata = 0
useWholeDB = False
if useWholeDB:
    con = sqlite3.connect('data/tweetsSpring.db')
    foursquaredata = pd.read_sql("SELECT * from tweets WHERE source = 'Foursquare'", con)
else:
    con = sqlite3.connect('data/foursquareSpring.db')
    foursquaredata = pd.read_sql("SELECT * from tweets", con)  

In [2]:
from bs4 import BeautifulSoup
def extract_urls(lst):
        urls = []
        for j in lst.split(' '):
            if j.startswith('http'):
                urls.append(j)
        return urls

def resolve_url(starturl):
    try:
        req = urllib2.Request(starturl)
        res = urllib2.urlopen(req, timeout = 2)
        finalurl = res.geturl()
        soup = BeautifulSoup(res.read())
        #print soup.find_all('script')
        #print(soup.prettify())
        
        return soup
    except:
        pass

In [4]:
print len(foursquaredata)

66342


In [ ]:
for iteration in range(2,6):
    print iteration
    htmls = []
    for index, row in foursquaredata[iteration*10000:(iteration+1)*10000].iterrows():
        lst = row.content
        urls = extract_urls(lst)
        if len(urls):
            htmls.append(resolve_url(urls[0]))
        progress(index-10000,10000)
        
    scripts = []
    for html in htmls:
        if (html is not None):
            if len(html.find_all('script'))>10:
                scripts.append([str(html.find_all('script')[10])])
            else:
                scripts.append([])
        else:
            scripts.append([])
            
    with open('data/script'+str(iteration+1)+'.p', 'wb') as fileToSave:
        np.save(fileToSave, scripts)

2
100.210000%

In [ ]:
redirects[0].find_all('script')[10]

In [3]:
    def progress(i, n):
        stdout.write("\r%f%%" % (i*100/float(n)))
        stdout.flush()
        if i == n-1:
            stdout.write("\r100%")
            print("\r\n")

In [5]:
len(htmls)

10000

In [14]:
scripts = []
for html in htmls:
    if (html is not None):
        if len(html.find_all('script'))>10:
            scripts.append([str(html.find_all('script')[10])])
        else:
            scripts.append([])
    else:
        scripts.append([])

In [17]:
with open('data/script2.p', 'wb') as fileToSave:
    np.save(fileToSave, scripts)

In [18]:
with open('data/script2.p', 'r') as fileToSave:
    theScripts = np.load(fileToSave)

In [19]:
print len(theScripts)

10000
